## Set up 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
PROJ_DIR = '/content/drive/MyDrive/MIDS/AIPI/Diginetica'
# change current directory after mounting
%cd $PROJ_DIR
! ls

/content/drive/MyDrive/MIDS/AIPI/Diginetica
cal_pop.py	      NextItNetModules.py  SA2C_v2new_dg.py  utility.py
data		      preprocessing_dg.py  SASRecModules.py
gen_replay_buffer.py  __pycache__	   split_train.py


In [ ]:
import os

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

## Download and unzip files from [here](https://competitions.codalab.org/competitions/11161#learn_the_details-data2)

In [ ]:
clicks = pd.read_csv('train-clicks.csv', sep = ';')
view = pd.read_csv('train-item-views.csv', sep = ';')
purchase = pd.read_csv('train-purchases.csv', sep = ';')
queries = pd.read_csv('train-queries.csv', sep = ';')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
purchase.head()

,sessionId,userId,timeframe,eventdate,ordernumber,itemId
0,150,18278.0,17100868,2016-05-06,16421,25911
1,151,NaN,6454547,2016-05-06,16290,175874
2,156,7.0,1721689387,2016-05-27,21173,35324
3,179,NaN,343001,2016-05-09,16924,31233
4,246,34.0,2311046,2016-05-09,16936,34677


In [ ]:
view.head()

,sessionId,userId,itemId,timeframe,eventdate
0,1,NaN,81766,526309,2016-05-09
1,1,NaN,31331,1031018,2016-05-09
2,1,NaN,32118,243569,2016-05-09
3,1,NaN,9654,75848,2016-05-09
4,1,NaN,32627,1112408,2016-05-09


In [ ]:
# this is probably used with combination with query data set 
clicks.head()

,queryId,timeframe,itemId
0,1,16338861,24857
1,46255,16404912,30792
2,46689,3831948,8252
3,46731,16273568,33969
4,46748,4058493,7837


## Data Preprocessing 


In [ ]:
# check how many records we have for the data that has user id 
print(len(purchase['userId'].notnull()))
print(len(view['userId'].notnull()))


18025
1235380


In [ ]:
# filter out the data where we don't have user id info
only_view = view[view['userId'].notnull()]

In [ ]:
only_view.head()

,sessionId,userId,itemId,timeframe,eventdate,eventdate_unix
164,48,2.0,24764,8863,2016-04-09,1.460160e+09
165,48,2.0,24764,496381,2016-04-09,1.460160e+09
166,48,2.0,24764,265216,2016-04-09,1.460160e+09
167,48,2.0,24764,519975,2016-04-09,1.460160e+09
168,48,2.0,24764,456437,2016-04-09,1.460160e+09


In [ ]:
# change the date to unix timestamp 
only_view['eventdate']= pd.to_datetime(only_view['eventdate'])
only_view['eventdate_unix'] = only_view['eventdate'].apply(lambda x: pd.Timestamp(x).timestamp())


<ipython-input-92-d86a2b162eea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_view['eventdate']= pd.to_datetime(only_view['eventdate'])
<ipython-input-92-d86a2b162eea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_view['eventdate_unix'] = only_view['eventdate'].apply(lambda x: pd.Timestamp(x).timestamp())


In [ ]:
# drop the columns we don't need 
only_view = only_view.copy().drop(['timeframe', 'eventdate', 'sessionId'], axis = 1)
# mark that the customer didn't buy 
only_view['is_buy'] = 0 

In [ ]:
# filter out the data where we don't have user id info
only_purchase= purchase[purchase['userId'].notnull()]
only_purchase.head()

,sessionId,userId,timeframe,eventdate,ordernumber,itemId
0,150,18278.0,17100868,2016-05-06,16421,25911
2,156,7.0,1721689387,2016-05-27,21173,35324
4,246,34.0,2311046,2016-05-09,16936,34677
7,322,167422.0,1085316,2016-05-25,20849,69167
8,365,87.0,377426862,2016-02-27,3072,10290


In [ ]:
# change the date to unix timestamp 
only_purchase['eventdate']= pd.to_datetime(only_purchase['eventdate'])
only_purchase['eventdate_unix'] = only_purchase['eventdate'].copy().apply(lambda x: pd.Timestamp(x).timestamp())


<ipython-input-103-6b6e7f213075>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_purchase['eventdate']= pd.to_datetime(only_purchase['eventdate'])
<ipython-input-103-6b6e7f213075>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_purchase['eventdate_unix'] = only_purchase['eventdate'].copy().apply(lambda x: pd.Timestamp(x).timestamp())


In [ ]:
only_purchase.head()

,sessionId,userId,timeframe,eventdate,ordernumber,itemId,eventdate_unix
0,150,18278.0,17100868,2016-05-06,16421,25911,1.462493e+09
2,156,7.0,1721689387,2016-05-27,21173,35324,1.464307e+09
4,246,34.0,2311046,2016-05-09,16936,34677,1.462752e+09
7,322,167422.0,1085316,2016-05-25,20849,69167,1.464134e+09
8,365,87.0,377426862,2016-02-27,3072,10290,1.456531e+09


In [ ]:
# drop the columns we don't need 
only_purchase = only_purchase.copy().drop(['timeframe', 'eventdate', 'ordernumber', 'sessionId'], axis = 1)
only_purchase['is_buy'] = 1

In [ ]:
# merge two data frame 
all_event = pd.concat([only_purchase, only_view], ignore_index=True, axis=0)
# check the data is correctly merged 
assert(only_purchase.shape[0] + only_view.shape[0] == all_event.shape[0])

In [ ]:
# create one hot encoded vectors 

item_encoder = LabelEncoder()
user_encoder= LabelEncoder()
all_event['itemId'] = item_encoder.fit_transform(all_event.itemId)
all_event['userId'] = user_encoder.fit_transform(all_event.userId)


In [ ]:
# rename column to be consistent with retail rocket dataset 
all_event.rename(columns = {'userId':'session_id', 'itemId':'item_id', 'eventdate_unix':'timestamp'}, inplace = True)
sorted_events = all_event.sort_values(by=['session_id', 'timestamp'])


In [ ]:
# save as df file 
sorted_events.to_pickle('diginetica_sorted_events.pkl')

In [ ]:
sorted_events.to_pickle('diginetica_sorted_events.df')

## Prepare to run the DRL Model



In [ ]:
# pre-process the files to the format we want 
# this is the same outcome as the ones in the final pickle cell above 
!python preprocessing_dg.py

preprocessing_dg.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_view['eventdate']= pd.to_datetime(only_view['eventdate'])
preprocessing_dg.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_view['eventdate_unix'] = only_view['eventdate'].apply(lambda x: pd.Timestamp(x).timestamp())
preprocessing_dg.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [ ]:
# split the data into training data and validation data 
!python split_train.py

In [ ]:
# test the generate replay buffer file for DRL model 
!python gen_replay_buffer.py

In [ ]:
# calculate item popularity 
!python cal_pop.py

0.0
1.0
2.0
3.0


In [1]:
# install packages 
!pip install pandas trfl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 104 kB 5.3 MB/s 


## Run the model


In [5]:
!python SA2C_v2new_dg.py --model=GRU --epoch=15

SA2C_v2new_dg.py:88: UserWarning: `tf.nn.rnn_cell.GRUCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.GRUCell`, and will be replaced by that in Tensorflow 2.0.
  tf.compat.v1.nn.rnn_cell.GRUCell(self.hidden_size),
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
2022-12-14 07:41:26.102425: E tensorflow/stream_executor/cuda/cud